In [1]:
import os
## Removed id , key below , needs to be added .....
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""

In [2]:
import configparser
import psycopg2
from sql_queries import copy_table_queries, insert_table_queries
from Manage_RS_Cluster import get_Cluster_Props

#Config
config = configparser.ConfigParser()
config.read('dwh.cfg')
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")




def insert_tables(cur, conn):
    for query in insert_table_queries:
        print("Running Query - {}".format(query))
        cur.execute(query)
        conn.commit()
        
def load_staging_tables(cur, conn):
    for query in copy_table_queries:
        cur.execute(query)
        conn.commit()

In [3]:
DWH_ENDPOINT, _ = get_Cluster_Props()
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(
                            DWH_ENDPOINT,
                            DWH_DB,
                            DWH_DB_USER,
                            DWH_DB_PASSWORD,
                            DWH_PORT
                            ))
cur = conn.cursor()

In [4]:
print("Load staging tables")
load_staging_tables(cur, conn)
print("Completed load of staging tables")

Load staging tables
Completed load of staging tables


In [5]:

print("Inserting into main tables")
insert_tables(cur, conn)

Inserting into main tables
Running Query - 
    INSERT INTO songplays (startTime, userId, level, songId, artistId, sessionId, location, userAgent) 
        SELECT
            TIMESTAMP 'epoch' + (e.ts/1000 * INTERVAL '1 second'),
            e.userId,
            e.level,
            s.song_id,
            s.artist_id,
            e.sessionId,
            e.location,
            e.userAgent
        FROM event_stg e
        JOIN song_stg s ON
            e.song = s.title AND
            e.artist = s.artist_name 
        WHERE
            e.page = 'NextSong'

Running Query - 
    INSERT INTO users 
        SELECT  DISTINCT (userId)
                userId,
                firstName,
                lastName,
                gender,
                level
        FROM event_stg
        WHERE userid is not null

Running Query - 
    INSERT INTO songs 
        SELECT DISTINCT (song_id)
            artist_id,
            title,
            artist_id,
            year,
            duration
    

In [ ]:
conn.close()